In [48]:
import pandas as pd
from tqdm import tqdm

In [60]:
data = pd.read_csv('dataset/dataset.csv')

In [61]:
data.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,XxYUnited Kingdom☺️
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [62]:
print(data.shape)

(541909, 8)


TASK 1

In [63]:
data.Quantity = data['Quantity'].apply(lambda x: int(x.strip('@')))
data.InvoiceDate  = pd.to_datetime(data.InvoiceDate)
data.UnitPrice = data['UnitPrice'].apply(lambda x: float(x.strip('Ww')))
data['CustomerID'] = data['CustomerID'].apply(lambda x: float(str(x).strip('&#')))
data['Country'] = data['Country'].apply(lambda x: x.strip('XxY☺️'))
data['InvoiceNo'] = data['InvoiceNo'].apply(lambda x: x.strip('ä'))
data['StockCode'] = data['StockCode'].apply(lambda x: x.strip('ö^'))


In [64]:
data.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [7]:
data['Description'][:5]

0      WHITE HANGING HEART T-LIGHT HOLDER
1                     WHITE METAL LANTERN
2          CREAM CUPID HEARTS COAT HANGER
3    $KNITTED UNION FLAG HOT WATER BOTTLE
4         $RED WOOLLY HOTTIE WHITE HEART.
Name: Description, dtype: object

In [8]:
print(data.duplicated().sum())
data.drop_duplicates(inplace = True)
print(data.duplicated().sum())

1631
0


In [9]:
data.isnull().sum() / data.shape[0] * 100

InvoiceNo       0.000000
StockCode       0.000000
Description     0.189717
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     25.000833
Country         0.000000
dtype: float64

In [10]:
data['Description'].fillna('No Description', inplace = True)
data['CustomerID'].dropna(inplace=True)


In [11]:
data.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [12]:
print((data.Quantity < 0).any())
print((data.Quantity < 0).any())


True
True


In [13]:
data = data.loc[data['Quantity'] > 0]
data = data.loc[data['UnitPrice'] > 0]

TASK 2

In [16]:
from pinecone import Pinecone, ServerlessSpec
from transformers import MarianMTModel, MarianTokenizer
import torch
from transformers import BertTokenizer, BertModel

In [17]:
pc = Pinecone(api_key="e3b33799-e836-42a0-ba81-8bff88e490df")
# pinecone.init(api_key="YOUR_PINECONE_API_KEY", environment="us-east-1")

In [85]:
descriptions = data['Description'].unique()
codes = list(range(len(descriptions)))

In [19]:
tokenizer = tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

C:\Users\USER\anaconda3\envs\pytorch_library\Lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
C:\Users\USER\anaconda3\envs\pytorch_library\Lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

C:\Users\USER\anaconda3\envs\pytorch_library\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
index = Pinecone.Index('stock-description')

In [22]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return embeddings

In [26]:
len(stock_codes)

528483

In [30]:
vectors = []
for code, desc in tqdm(zip(stock_codes, descriptions)):
    embedding = get_embeddings(desc)
    vectors.append({'stockcode': str(code), 'description': embedding})

303it [1:00:03, 11.89s/it]


KeyboardInterrupt: 

In [ ]:
index.upsert(vectors)


In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(data.Description)
first_product_vector = tfidf_matrix[0].toarray().flatten()


In [ ]:
# index_name = 'stock-description'
# index = pc.Index("stock-description")
# pc.create_index(
#     name=index_name,
#     dimension=7805,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     ) 
# )



In [ ]:
query = data['Description'].iloc[5]
query_embed = tfidf_vectorizer.transform([query])
embed = query_embed.toarray()
results = index.query(embed, top_k = 5)
for result in results:
    print(result['id'], result['score'])

TASK 3

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

user_user_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix))
user_user_sim_matrix

In [31]:
def compute_similarity(embedding, embeddings, metric='cosine'):
    if metric == 'cosine':
        scores = cosine_similarity([embedding], embeddings)[0]
    elif metric == 'dot_product':
        scores = linear_kernel([embedding], embeddings)[0]
    return scores

query_text = "tea set ceramic"
query_embedding = get_embeddings(query_text)

cosine_scores = compute_similarity(query_embedding, embeddings, metric='cosine')
dot_product_scores = compute_similarity(query_embedding, embeddings, metric='dot_product')

print("Cosine Similarity Scores:", cosine_scores)
print("Dot Product Scores:", dot_product_scores)


NameError: name 'embeddings' is not defined

ENDPOINT

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List

In [ ]:
def generate_query_embedding(query):
    inputs = tokenizer(query, return_tensors='pt', truncation=True, padding='max_length', max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return embedding

In [ ]:
import pinecone
def query_pinecone(embedding, top_k=5):
    response = index.query(queries=[embedding.tolist()], top_k=top_k)
    return response['matches']

In [ ]:
def formulate_response(matches):
    if not matches:
        return "Sorry, we couldn't find any products matching your query."
    
    product_descriptions = [f"StockCode: {match['id']}, Score: {match['score']:.2f}" for match in matches]
    response = "Here are some products you might like:\n" + "\n".join(product_descriptions)
    return response

In [ ]:
app = FastAPI()
class QueryRequest(BaseModel):
    query: str
class QueryResponse(BaseModel):
    matches: List[dict]
    response: str
        
@app.post("/recommendations", response_model=QueryResponse)
async def recommend_products(request: QueryRequest):
    query = request.query
        query_embedding = generate_query_embedding(query)
        matches = query_pinecone(query_embedding)
        response_text = formulate_response(matches)
        output = {
        "matches": matches,
        "response": response_text}
    
    return output

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)